In [1]:
!pip install paddlepaddle-gpu
!wget https://ernie.bj.bcebos.com/ERNIE_Base_en_stable-2.0.0.tar.gz
!gunzip ERNIE_Base_en_stable-2.0.0.tar.gz
!tar -xvf ERNIE_Base_en_stable-2.0.0.tar
!git clone https://github.com/PaddlePaddle/ERNIE.git

     |████████████████████████████████| 236.1MB 1.2MB/s 
     |████████████████████████████████| 901kB 37.6MB/s 
     |████████████████████████████████| 24.8MB 8.0MB/s 
     |████████████████████████████████| 112kB 52.5MB/s 
  Stored in directory: /root/.cache/pip/wheels/dc/84/da/8aff50941f548db5384b076d5a6a6afea0cd12672e0326edc4
Successfully built rarfile
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
ERROR: paddlepaddle-gpu 1.5.1.post107 has requirement matplotlib<=2.2.4, but you'll have matplotlib 3.0.3 which is incompatible.
  Found existing installation: scipy 1.3.0
    Uninstalling scipy-1.3.0:
      Successfully uninstalled scipy-1.3.0
--2019-07-31 16:38:57--  https://ernie.bj.bcebos.com/ERNIE_Base_en_stable-2.0.0.tar.gz
Resolving ernie.bj.bcebos.com (ernie.bj.bcebos.com)... 103.235.46.61
Connecting to ernie.bj.bcebos.com (ernie.bj.bcebos.com)|103.235.46.61|:443... connected.
HTTP request sent, awaiting resp

## Import Packages

In [0]:
import pandas as pd
import os
import re

## Downloading and Preparing Dataset

In [3]:
!wget !wget https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip
!unzip 'drugsCom_raw.zip'

--2019-07-31 16:40:56--  http://!wget/
Resolving !wget (!wget)... failed: Name or service not known.
wget: unable to resolve host address ‘!wget’
--2019-07-31 16:40:56--  https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42989872 (41M) [application/x-httpd-php]
Saving to: ‘drugsCom_raw.zip’

drugsCom_raw.zip    100%[===================>]  41.00M  49.9MB/s    in 0.8s    

2019-07-31 16:40:57 (49.9 MB/s) - ‘drugsCom_raw.zip’ saved [42989872/42989872]

FINISHED --2019-07-31 16:40:57--
Total wall clock time: 1.1s
Downloaded: 1 files, 41M in 0.8s (49.9 MB/s)
Archive:  drugsCom_raw.zip
  inflating: drugsComTest_raw.tsv    
  inflating: drugsComTrain_raw.tsv   


In [4]:
df = pd.read_csv('drugsComTrain_raw.tsv', delimiter='\t')
df = df[['review', 'rating']]
df.columns = ['text_a', 'label']
df['label'] = df['label'].apply(lambda x: int(x))
df['label'] = df['label'].apply(lambda x: 1 if int(x) > 5 else 0)
df['text_a'] = df['text_a'].apply(lambda x: re.sub('[^a-zA-Z]', ' ', str(x)))
!mkdir -p 'dataset/SST-2'
train = df.iloc[:, :int(len(df)*0.8)]
dev = df.iloc[:, int(len(df)*0.8): ]
train.to_csv('train.tsv', index=False, sep='\t')
dev.to_csv('dev.tsv', index=False, sep='\t')
print("saved")

saved


In [7]:
# Let's prepare test data also

test = pd.read_csv('drugsComTest_raw.tsv', sep='\t')
test = test[['review']]
test['review'] = test['review'].apply(lambda x: re.sub('[^a-zA-Z]', ' ', str(x)))
test.to_csv('test.tsv', index=False, sep='\t')
print("saved test file")

saved test file


## Get environment ready for train and testing

### -create new folder for dataset
### -set path for model parameters and dataset

In [0]:
!mkdir -p paramters/params
!mv train.tsv dataset/SST-2/
!mv dev.tsv dataset/SST-2/
!mv test.tsv dataset/SST-2/
!mv params/ parameters/params

In [10]:
!mv dataset/ ERNIE/
!mv -r parameters/ ERNIE/

mv: cannot stat 'dataset/': No such file or directory


## Start Model Training

In [32]:
os.chdir('ERNIE/')
os.environ['TASK_DATA_PATH']='dataset'
os.environ['MODEL_PATH']='parameters'
print(os.getcwd())
!sh script/en_glue/ernie_base/SST-2/task.sh

/content/ERNIE
script/en_glue/ernie_base/SST-2/task.sh: 7: script/en_glue/ernie_base/SST-2/task.sh: [[: not found
-----------  Configuration Arguments -----------
batch_size: 32
checkpoints: ./checkpoints
dev_set: dataset/SST-2/dev.tsv
diagnostic: None
diagnostic_save: None
do_lower_case: True
do_test: True
do_train: True
do_val: True
doc_stride: 128
enable_ce: False
epoch: 4
ernie_config_path: script/en_glue/ernie_base/ernie_config.json
for_cn: False
in_tokens: False
init_checkpoint: None
init_pretraining_params: parameters/params
is_classify: True
is_regression: False
label_map_config: None
learning_rate: 2e-05
loss_scaling: 1.0
lr_scheduler: linear_warmup_decay
max_answer_length: 100
max_query_length: 64
max_seq_len: 128
metric: simple_accuracy
metrics: True
n_best_size: 20
num_iteration_per_drop_scope: 1
num_labels: 2
predict_batch_size: None
random_seed: 1
save_steps: 10000
shuffle: True
skip_steps: 10
task_id: 0
test_save: output/test_out.{1..5}.2e-5.32.4.tsv
test_set: dataset/SS